## Obtenir les valeurs de références pour les calculs

In [1]:
def get_params(self):
    '''
    -------------------------------------------------------
    
    Aim : Function which get the refered matrix
    parameters and refered mean values of source term
    and hydraulic conductivity
    
    -------------------------------------------------------
    
    Valeur input : 
        - nothing
        
    Valeur output :
        - nothing
        
    -------------------------------------------------------
    '''

    # initialisation of the covariance model and define a reference random field array for hydraulic conductivity
    cov_mod = gn.covModel.CovModel2D(self.el, self.alpha, 'title')
    geosclassic_output = gn.geosclassicinterface.simulate2D(cov_mod, (self.nbx_nd, self.nby_nd), (self.dx, self.dy), (self.ox, self.oy), 
                                                            mean=1.8, var=0.5**2,
                                                            method='simple_kriging', 
                                                            searchRadiusRelative=1.0, nneighborMax=12,
                                                            nreal=1, seed=8948474, verbose=0)

    # save the refence random field  
    self.k_matrix = geosclassic_output['image'].val[0][0]

    # define a reference random field array for groundwater recharge (or source/sink)
    geosclassic_output = gn.geosclassicinterface.simulate2D(cov_mod, (self.nbx_nd, self.nby_nd), (self.dx, self.dy), (self.ox, self.oy), 
                                                            mean=400, var=15**2,
                                                            method='simple_kriging', 
                                                            searchRadiusRelative=1.0, nneighborMax=12,
                                                            nreal=1, seed=7528660, verbose=0)

    # save the reference random field
    self.r_matrix = geosclassic_output['image'].val[0][0]/365/1000
    
    # save the mean values of hydraulic conductivity (k_mean), groundwater recharge (r_mean), thickness (b), porosity (poro)
    self.k_mean = gmean(self.k_matrix, axis=None)
    self.r_mean = np.mean(self.r_matrix, axis=None)
    self.b = self.doc.getParamValue(108, 0) - self.doc.getParamValue(106, 0)
    self.poro = self.doc.getParamValue(201, 0)

## Initier les valeurs des paramètres dans Feflow et simulation numérique

In [5]:
def set_parameters_and_simulation(self, list_id_parms, list_val_parms):
    '''
    -------------------------------------------------------
    
    Aim : Function which set the values of a or several
    parameters in the Feflow file. After a Feflow
    simulation is done
    
    -------------------------------------------------------
    
    Valeur input : 
        - list_id_params : list of number corresponding
        to the Id of the parameters [-]

        - list_val_params : list of values of each
        parameters [-]
        
    Valeur output :
        - nothing
        
    -------------------------------------------------------
    '''
    
    # put at i-th position the i-th value of the j-th id/parameter in feflow file
    for j in range(len(list_id_parms)):
        for i in range(self.nelements):
            self.doc.setParamValue(list_id_parms[j], i, list_val_parms[j][i])
    
    # start and stop a feflow simulation
    self.doc.startSimulator()
    self.doc.stopSimulator()

## Obtenir les valeurs des simulations numériques

In [4]:
def get_results(self):
    '''
    -------------------------------------------------------
    
    Aim : Function which get the values of the results 
    (hydraulic charge, age, exit-time, capture
    probabilities, contributions) in a dictionnary
    
    -------------------------------------------------------
    
    Valeur input : 
        - nothing
        
    Valeur output :
        - nothing
        
    -------------------------------------------------------
    '''
    
    # constant value juste to upgrade the index value
    var = 1
    
    # for loop to get values after the Feflow simulation
    for i in self.dict_results:
        if i=='charge':
            self.dict_results[i] = np.array([ self.doc.getResultsFlowHeadValue(i) for i in range(self.nnodes) ])
            self.dict_results[i].shape = (self.nby_nd, self.nbx_nd)
        
        elif i=='age' or i=='exit-time':
            self.doc.setMultiSpeciesId(var)
            self.dict_results[i] = np.array([ self.doc.getResultsTransportMassValue(i) for i in range(self.nnodes) ])
            self.dict_results[i][ self.dict_results[i]<0 ] = 0
            self.dict_results[i].shape = (self.nby_nd, self.nbx_nd)
            var+=1

        elif i=='probability':
            self.doc.setMultiSpeciesId(var)
            self.dict_results[i] = np.array([ self.doc.getResultsTransportMassValue(i) for i in range(self.nnodes) ])
            self.dict_results[i][ self.dict_results[i]<0 ] = 0
            self.dict_results[i][ self.dict_results[i]>1] = 1
            self.dict_results[i].shape = (self.nby_nd, self.nbx_nd)
            var+=1
        
        elif i=='contributions':
            self.dict_results[i] = self.dict_results['probability'] * self.dict_params['rech'] * self.s
            self.dict_results[i].shape = (self.nby_nd, self.nbx_nd)

## Obtenir les autres paramètres du modèle numérique 

In [3]:
def get_constants(self, list_of_id):
    '''
    -------------------------------------------------------
    
    Aim : Function which get the values of the constants
    parameters from the Feflow files
    
    -------------------------------------------------------
    
    Valeur input : 
        - list_of_id : list of number corresponding to
        the Id of the parameter in Feflow files [-]
        
    Valeur output :
        - nothing
        
    -------------------------------------------------------
    '''

    # title of the DataFrame
    tab = [r'$A$ [-]', r'$\theta$ [deg]', r'$z_{t}$ [m]', r'$z_{b}$ [m]', r'$S$ [-]', r'$\alpha$ [-]', r'$K$ [-]', r'$\lambda$ [1/d]', r'$D$ [m2/s]', r'$d_{l}$ [m]', r'$d_{t}$ [m]']
    
    # for loop to put constant values in the DataFrame
    for i in range(len(list_of_id)):
        self.dtframe_constant[tab[i]] = [self.doc.getParamValue(list_of_id[i], 0)]

## Choisir les cas d'études

In [6]:
def set_variation_homogeneity(self, conditional=False, perm_sim=False, rech_sim=False):
    '''
    -------------------------------------------------------
    
    Aim : Function which set the case for the study
    methods and call the "set_parameters_and_simulation"
    method to put values in Feflow file for the simulation
    
    -------------------------------------------------------
    
    Valeur input : 
        - conditional : boolean variable which determine
        if conditional values is used or not [-]

        - perm_sim : boolean variable which determine
        if hydraulic conductivity is random or not

        - rech_sim : boolean variable which determine
        if groundwater recharge is random or not
        
    Valeur output :
        - nothing
        
    -------------------------------------------------------
    '''
    
    # Test to determine the case of homogeneity of parameters (recharge and permeability)
    # and if the values used are conditional or unconditional.
    if conditional==True:
        cov_model = gn.covModel.CovModel2D(self.el, self.alpha, 'title')            
        self.dict_params['perm'] = self.cond_val(cov_model, self.nbx_nd, self.nby_nd, self.dx, self.dy, self.ox, self.oy, self.pos_cond, self.v1, 1.8, 0.5)
        self.dict_params['rech'] = self.cond_val(cov_model, self.nbx_nd, self.nby_nd, self.dx, self.dy, self.ox, self.oy, self.pos_cond, self.v2, 1.1e-3, 1e-4)
        
    else:
        if perm_sim==True:
            self.dict_params['perm'] = self.k_matrix
        else:
            self.dict_params['perm'] = np.ones((self.nby_nd, self.nbx_nd))*self.k_mean
            
        if rech_sim==True:
            self.dict_params['rech'] = self.r_matrix
        else:
            self.dict_params['rech'] = np.ones((self.nby_nd, self.nbx_nd))*self.r_mean

    # get parameters values for element matrix
    k_el = self.dict_params['perm'][:self.nby_el, :self.nbx_el].flatten()
    r_el = self.dict_params['rech'][:self.nby_el, :self.nbx_el].flatten()
    
    # Set parameters values to Feflow by the function "set_parameters_and_simulation"
    # and simulation this case
    self.set_parameters_and_simulation([100, 113], [10**k_el,  r_el])